# Loop 50 Strategic Analysis

## Current Situation
- Best CV: 0.008194 (exp_032)
- Best LB: 0.0877 (exp_030)
- Target: 0.0347
- Gap: 152.7%

## Key Question
The GroupKFold experiment (exp_051) showed CV 0.008807 (7.5% worse than Leave-One-Out).
This confirms that the validation scheme is NOT the cause of the CV-LB gap.

## Analysis Goals
1. Analyze the CV-LB relationship across all submissions
2. Identify if any approach has a different CV-LB relationship
3. Determine what strategies might reduce the intercept

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# All submissions with CV and LB scores
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982, 'model': 'MLP'},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065, 'model': 'LGBM'},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972, 'model': 'MLP+DRFP'},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969, 'model': 'Ensemble'},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946, 'model': 'Simpler MLP'},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932, 'model': 'Even Simpler'},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936, 'model': 'Ridge'},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913, 'model': 'Simple Ensemble'},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893, 'model': 'ACS PCA'},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887, 'model': 'Weighted Loss'},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877, 'model': 'GP+MLP+LGBM'},
    {'exp': 'exp_041', 'cv': 0.0090, 'lb': 0.0932, 'model': 'Aggressive Reg'},
    {'exp': 'exp_042', 'cv': 0.0145, 'lb': 0.1147, 'model': 'Pure GP'},
]

df = pd.DataFrame(submissions)
print(f"Total submissions: {len(df)}")
print(df)

In [ ]:
# Fit linear regression: LB = slope * CV + intercept
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])

print(f"\n=== CV-LB Relationship ===")
print(f"Linear fit: LB = {slope:.2f} × CV + {intercept:.4f}")
print(f"R² = {r_value**2:.4f}")
print(f"Intercept = {intercept:.4f}")
print(f"Target LB = 0.0347")
print(f"\nIntercept ({intercept:.4f}) > Target (0.0347): {intercept > 0.0347}")

# Required CV to hit target
required_cv = (0.0347 - intercept) / slope
print(f"\nRequired CV to hit target: {required_cv:.6f}")
if required_cv < 0:
    print("⚠️ IMPOSSIBLE: Required CV is NEGATIVE!")
    print("The current paradigm CANNOT reach the target.")

In [ ]:
# Plot CV vs LB
plt.figure(figsize=(10, 6))
plt.scatter(df['cv'], df['lb'], c='blue', s=100, alpha=0.7)

# Add labels
for i, row in df.iterrows():
    plt.annotate(row['exp'], (row['cv'], row['lb']), fontsize=8, ha='left')

# Fit line
cv_range = np.linspace(0, 0.02, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'LB = {slope:.2f}×CV + {intercept:.4f}')

# Target line
plt.axhline(y=0.0347, color='green', linestyle=':', label='Target LB = 0.0347')

# Intercept line
plt.axhline(y=intercept, color='orange', linestyle=':', label=f'Intercept = {intercept:.4f}')

plt.xlabel('CV Score')
plt.ylabel('LB Score')
plt.title('CV vs LB Relationship (All Submissions)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_relationship.png', dpi=150)
plt.show()

print(f"\n=== KEY INSIGHT ===")
print(f"The intercept ({intercept:.4f}) is HIGHER than the target (0.0347).")
print(f"This means even with CV=0, the expected LB would be {intercept:.4f}.")
print(f"The current paradigm CANNOT reach the target.")

In [ ]:
# Analyze residuals - are any models off the line?
df['predicted_lb'] = slope * df['cv'] + intercept
df['residual'] = df['lb'] - df['predicted_lb']
df['residual_pct'] = df['residual'] / df['predicted_lb'] * 100

print("\n=== Residual Analysis ===")
print("Looking for models that deviate from the CV-LB line...")
print(df[['exp', 'model', 'cv', 'lb', 'predicted_lb', 'residual', 'residual_pct']].to_string())

print(f"\n=== Summary ===")
print(f"Mean absolute residual: {df['residual'].abs().mean():.4f}")
print(f"Max positive residual: {df['residual'].max():.4f} ({df.loc[df['residual'].idxmax(), 'exp']})")
print(f"Max negative residual: {df['residual'].min():.4f} ({df.loc[df['residual'].idxmin(), 'exp']})")

print(f"\n=== Conclusion ===")
print("All models fall on the SAME CV-LB line (R²=0.98).")
print("No model type has a fundamentally different relationship.")
print("The problem is STRUCTURAL distribution shift, not model choice.")

In [ ]:
# What would it take to reach the target?
print("\n=== WHAT WOULD IT TAKE TO REACH TARGET? ===")
print(f"Target LB: 0.0347")
print(f"Current intercept: {intercept:.4f}")
print(f"Current slope: {slope:.2f}")

print(f"\n--- Option 1: Reduce Intercept ---")
required_intercept = 0.0347 - slope * 0.008  # Assume best CV ~0.008
print(f"If CV stays at 0.008, need intercept = {required_intercept:.4f}")
print(f"Current intercept: {intercept:.4f}")
print(f"Reduction needed: {intercept - required_intercept:.4f} ({(intercept - required_intercept)/intercept*100:.1f}%)")

print(f"\n--- Option 2: Reduce Slope ---")
required_slope = (0.0347 - intercept) / 0.008  # Assume best CV ~0.008
print(f"If intercept stays at {intercept:.4f}, need slope = {required_slope:.2f}")
print(f"Current slope: {slope:.2f}")
print(f"This would require NEGATIVE slope (impossible with current paradigm)")

print(f"\n--- Option 3: Both ---")
print(f"Need to find an approach with DIFFERENT CV-LB relationship")
print(f"Possible strategies:")
print(f"  1. Uncertainty-weighted predictions (blend toward mean when extrapolating)")
print(f"  2. Solvent clustering (class-specific models)")
print(f"  3. Pre-trained molecular embeddings (transfer learning)")
print(f"  4. Physics-informed constraints (Arrhenius, thermodynamics)")
print(f"  5. Pseudo-labeling (use confident test predictions to adapt)")

In [ ]:
# Analyze the best experiments
print("\n=== BEST EXPERIMENTS ===")
best_cv = df.loc[df['cv'].idxmin()]
best_lb = df.loc[df['lb'].idxmin()]

print(f"Best CV: {best_cv['exp']} with CV={best_cv['cv']:.4f}, LB={best_cv['lb']:.4f}")
print(f"Best LB: {best_lb['exp']} with CV={best_lb['cv']:.4f}, LB={best_lb['lb']:.4f}")

print(f"\n=== UNEXPLORED DIRECTIONS ===")
print("After 51 experiments, we've tried:")
print("  ✓ MLP (various architectures)")
print("  ✓ LightGBM, XGBoost, CatBoost, Random Forest")
print("  ✓ Gaussian Process")
print("  ✓ Ridge Regression")
print("  ✓ Ensembles (GP+MLP+LGBM)")
print("  ✓ Various feature sets (Spange, DRFP, ACS PCA)")
print("  ✓ Regularization strategies")
print("  ✓ GroupKFold validation")

print("\nWhat HASN'T been tried:")
print("  ✗ Uncertainty-weighted predictions")
print("  ✗ Extrapolation detection features")
print("  ✗ Pseudo-labeling / test-time adaptation")
print("  ✗ Solvent clustering / class-specific models")
print("  ✗ Pre-trained molecular embeddings (ChemBERTa, MolBERT)")
print("  ✗ Graph Neural Networks (but may not be allowed)")

In [ ]:
# Final recommendation
print("\n" + "="*60)
print("STRATEGIC RECOMMENDATION")
print("="*60)

print("\n1. DO NOT SUBMIT exp_051 (GroupKFold)")
print("   - CV is 7.5% worse than best")
print("   - No evidence of different CV-LB relationship")

print("\n2. FOCUS ON REDUCING THE INTERCEPT")
print("   - The intercept (0.0533) > target (0.0347)")
print("   - Need to reduce intercept by ~35%")
print("   - This requires fundamentally different approach")

print("\n3. RECOMMENDED NEXT EXPERIMENTS:")
print("   a) Uncertainty-weighted predictions")
print("      - Use GP uncertainty to blend toward mean")
print("      - May reduce extreme predictions on unseen solvents")
print("   b) Extrapolation detection")
print("      - Add features measuring distance to training distribution")
print("      - Conservative predictions when extrapolating")
print("   c) Optuna hyperparameter optimization")
print("      - The 'mixall' kernel uses Optuna")
print("      - May find better weight combinations")

print("\n4. SUBMISSION STRATEGY (5 remaining):")
print("   - Save submissions for approaches with DIFFERENT CV-LB relationship")
print("   - Don't submit incremental improvements on same line")
print("   - Each submission should test a fundamentally different hypothesis")